In [1]:
import torch
import torch.nn as nn
import torchvision

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import os

from torch.utils.data import Dataset, DataLoader
# from torch.utils.data.sampler import Sampler
import torch.optim as optim
import sys
sys.path.append('../')

from dataset import LbpDataset, train_transforms, val_transforms, test_transforms, collate_fn, get_data
from visualize import visualize
from rcnn_model import fasterrcnn_resnet201_fpn, FastRCNNPredictor
from engine import evaluate
import utils
from train_lbp import get_train_test_list

In [2]:
from torchvision.models.detection.retinanet import retinanet_resnet50_fpn

In [3]:
import easydict 
args = easydict.EasyDict({ "batch_size": 8, 
                          "epochs": 50, 
                          "data": 0, 
                          'lr':0.1,
                         'momentum':0.9,
                         'weight_decay':1e-4,
                         'start_epoch':0,
                         'gpu':5,
                          'workers':2,
                         'output_dir' :'../trained_model/'})

In [4]:
df = pd.read_csv('../../data/df.csv')
df.head()
# Data loading code
data_dir = '../../data/df.csv'
train_list, test_list = get_train_test_list(data_dir)
train_dataset = LbpDataset(train_list, transform=train_transforms)
test_dataset = LbpDataset(test_list, transform=val_transforms)  

total 4019 train 3014 test 1005
3014
1005


In [5]:
train_sampler = torch.utils.data.RandomSampler(train_dataset)
test_sampler = torch.utils.data.SequentialSampler(test_dataset)

train_loader = DataLoader(
    train_dataset, batch_size=args.batch_size,
    sampler=train_sampler, num_workers=args.workers,
    collate_fn=utils.collate_fn)

test_loader = DataLoader(
    test_dataset, batch_size=args.batch_size,
    sampler=test_sampler, num_workers=args.workers,
    collate_fn=utils.collate_fn)

In [6]:
num_classes = 2
pretrained = retinanet_resnet50_fpn(pretrained=True)
model = retinanet_resnet50_fpn(pretrained=False, min_size=1024, max_size=1024, num_classes=num_classes)
model.backbone.load_state_dict(pretrained.backbone.state_dict())
model.head.regression_head.load_state_dict(pretrained.head.regression_head.state_dict())
device = torch.device('cuda')
model.to(device)
print('model is loaded to gpu')

model is loaded to gpu


In [7]:
# # pretrained.state_dict()
# for p in model.parameters() :
#     if p.requires_grad == True :
#         print(p.shape)

In [8]:
# model.head.regression_head

In [9]:
params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.Adam(params, lr=0.0001)
optimizer = torch.optim.SGD(
       params, lr=0.001, momentum=0.9, weight_decay=1e-4)
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20, 40, 60, 80, 100], 
                                                    gamma=0.5)

In [10]:
from train_lbp import train_one_epoch

start_time = time.time()
for epoch in range(120):
    train_one_epoch(model, optimizer, train_loader, device, epoch, 400)
    lr_scheduler.step()
    
    if epoch > 20 and epoch % 5 == 0 :
        if args.output_dir:
            checkpoint = {
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'lr_scheduler': lr_scheduler.state_dict(),
                'args': args,
                'epoch': epoch
            }
            utils.save_on_master(
                checkpoint,
                os.path.join(args.output_dir, 'model_{}.pth'.format(epoch)))
            utils.save_on_master(
                checkpoint,
                os.path.join(args.output_dir, 'checkpoint.pth'))

        # evaluate after every epoch
        evaluate(model, test_loader, device=device)    
print('total time is {}'.format(time.time() - start_time))    

Epoch: [0]  [  0/377]  eta: 0:13:41  lr: 0.001000  loss: 3.7088 (3.7088)  classification: 1.1390 (1.1390)  bbox_regression: 2.5697 (2.5697)  time: 2.1788  data: 1.3296  max mem: 7428
Epoch: [0]  [376/377]  eta: 0:00:00  lr: 0.001000  loss: 1.4234 (1.7307)  classification: 0.8647 (0.9300)  bbox_regression: 0.5700 (0.8006)  time: 0.8612  data: 0.0182  max mem: 7677
Epoch: [0] Total time: 0:05:29 (0.8740 s / it)
Epoch: [1]  [  0/377]  eta: 0:13:40  lr: 0.001000  loss: 1.4662 (1.4662)  classification: 0.8605 (0.8605)  bbox_regression: 0.6057 (0.6057)  time: 2.1767  data: 1.3238  max mem: 7677
Epoch: [1]  [376/377]  eta: 0:00:00  lr: 0.001000  loss: 1.4382 (1.5008)  classification: 0.8634 (0.8720)  bbox_regression: 0.5785 (0.6287)  time: 0.8609  data: 0.0185  max mem: 7677
Epoch: [1] Total time: 0:05:30 (0.8764 s / it)
Epoch: [2]  [  0/377]  eta: 0:13:46  lr: 0.001000  loss: 1.4821 (1.4821)  classification: 0.8681 (0.8681)  bbox_regression: 0.6140 (0.6140)  time: 2.1926  data: 1.3421  max m

In [11]:
evaluate(model, test_loader, device=device) 

creating index...
index created!
Test:  [  0/126]  eta: 0:03:37  model_time: 0.3885 (0.3885)  evaluator_time: 0.0259 (0.0259)  time: 1.7263  data: 1.2843  max mem: 7677
Test:  [100/126]  eta: 0:00:14  model_time: 0.3605 (0.3591)  evaluator_time: 0.0263 (0.0278)  time: 0.5599  data: 0.1582  max mem: 7677
Test:  [125/126]  eta: 0:00:00  model_time: 0.3614 (0.3584)  evaluator_time: 0.0249 (0.0275)  time: 0.5462  data: 0.1525  max mem: 7677
Test: Total time: 0:01:11 (0.5635 s / it)
Averaged stats: model_time: 0.3614 (0.3584)  evaluator_time: 0.0249 (0.0275)
Accumulating evaluation results...
DONE (t=0.54s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.072
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.163
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.048
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=med